In [30]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-128079")
exp = Experiment(workspace=ws, name="quick-starts-ws-128079")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-128079
Azure region: southcentralus
Subscription id: 572f8abf-a1a0-4b78-8c6d-3630739c72b5
Resource group: aml-quickstarts-128079


In [31]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# choose a name for your cluster
cluster_name= "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws , name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2' , max_nodes=4)
    #create the cluster
    cpu_cluster=ComputeTarget.create(ws,cluster_name,compute_config)



print('cpu-cluster has been created')

Found existing compute target
cpu-cluster has been created


In [32]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps =RandomParameterSampling({
    "--C": uniform(0.6,1),
    "--max_iter": uniform(100,110)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#if "training" not in os.listdir():
   # os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est =SKLearn(source_directory='./',
             entry_script='train.py',
             compute_target='cpu-cluster',    
             vm_size='Standard_D2_V2'
             )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4 
                                    
                                    )

In [33]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run=exp.submit(config=hyperdrive_config)


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [34]:
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [36]:
#To periodically check the status of the run object, and navigate to Azure portal to monitor the run
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-128079,HD_23bb988c-18ca-42c5-8444-bff8fa2257f7,hyperdrive,Canceled,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())

In [ ]:
#save the best model
model =best_run.register_model(model_name = 'best_model', model_path='model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
web_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = Dataset.Tabular.from_delimited_files(path=web_path)
ds.head()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)



In [ ]:
import pandas as pd 
import sklearn.model_selection import train_test_split 
from sklearn import datasets
from azureml.core.dataset import Dataset
import numpy as np 
from sklearn import preprocessing

#split the dataset into train and test
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)


if not os.path:isdir('dataset'):
    os.mkdir('dataset')

    training_data.to_csv('dataset/y.csv')
    datastore= ws.get_defult_datastore()
    datastore.upload(scr_dir='/dataset', target_path='ydataset', overwrite=True,show_progress=True)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###